In [1]:
from langchain import LLMMathChain, OpenAI, SerpAPIChain
from langchain.chains.jurassic_x.base import JurassicXChain, ChainConfig

In [2]:
llm = OpenAI(temperature=0)
search = SerpAPIChain()
llm_math_chain = LLMMathChain(llm=llm)
chains = [
    ChainConfig(
        action_name = "Search",
        action=search.search,
        action_description="useful for when you need to answer questions about current events"
    ),
    ChainConfig(
        action_name="Calculator",
        action=llm_math_chain.run,
        action_description="useful for when you need to answer questions about math"
    )
]

In [3]:
jchain = JurassicXChain.from_chains(llm, chains, verbose=True)

In [7]:
jchain.run("What is the age of Kim Kardashian's boyfriend raised to the 0.23 power?")

 I need to find out how old Kim Kardashian's boyfriend is.
Action: Search
Action Input: "Kim Kardashian's boyfriend"
Kim Kardashian's boyfriend Pete Davidson has revealed his hopes to one day become a dad, admitting he's 'so excited' to have children.
 I need to find out how old Pete Davidson is.
Action: Search
Action Input: "Pete Davidson age"
28 years
 I need to calculate 28 to the 0.23 power
Action: Calculator
Action Input: 28^0.23
Answer: 2.1520202182226886

 I now know the final answer
Final Answer: 2.1520202182226886


'2.1520202182226886'